In [4]:
from img2table.document import Image
import io
from PIL import Image as PIL_Image, ImageDraw
import numpy as np
import cv2
import pandas as pd

In [5]:
def classify_image(image, model):
    """
    Classify the image to numbers from 0 to 8, 8 is None
    
    Parameter
    ---------
    image: ndarray of a single channel image.
    
    Return
    ------
    pred: classified value corresponding to input image
    
    """
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    resized = cv2.resize(gray, (40, 40))  # 40x40 pixels is the input shape of model

    # Expand the dimensions of the image to match the input shape of the model
    im = np.expand_dims(resized, axis=0)

    result = model.predict(im)
    pred = np.argmax(result[0])

    return pred

In [6]:
def cell_extraction_classification_df_return(orddict, img, model):
    """
    Extracts cells of the image from the bbox values in orddict,
    classify the image using our custom ocr model, and returns the result as a dataframe
    
    Parameter
    ---------
    orddict: ordered Dictionary having 4 values of bbox
    
    Return
    ------
    df: dataframe of classified values
    
    """
    del orddict[0], orddict[4]  # del the first and last keys (rows) of orddict
    pred = []
    for key, cell_list in orddict.items():  # do the bbox extrn and classification using our model
        for cell in cell_list:
            x1 = cell.bbox.x1
            y1 = cell.bbox.y1
            x2 = cell.bbox.x2
            y2 = cell.bbox.y2

            new_im = img.crop((x1, y1, x2, y2))
            im_arr = np.array(new_im)  # Converting new_im (PIL.Image.Image) to numpy array for predict_image()
            pred.append(classify_image(im_arr, model))
    pred = [0 if num == 8 else num for num in pred]
    pred_arr = np.array(pred)
    reshaped_pred_arr = pred_arr.reshape(3, 13)  # 3 rows and 13 columns
    df = pd.DataFrame(reshaped_pred_arr)
    return df

In [7]:
def dataframe_postprocessing(paper_df):
    """
    Preprocessing the dataframe - removing first column,
    flattening the np.array of df column-wise,
    and returns the values to be added to main mark-dictionary
        
    Parameter
    ---------
    paper_df: Output of cell_extraction_classification_df_return(), df with the unwanted first column
    
    Return
    ------
    cell_vals: Values to be added to main mark-dictionary
    
    """
    paper_df = paper_df.iloc[:, 1:]  # iloc[row, column], removing first column

    # Flattening & adding marks to my_dict
    paper_arr = paper_df.to_numpy()
    flat = paper_arr.flatten(order='F')  # F - flattening column-wise
    cell_vals = [i for i in flat]

    return cell_vals

In [8]:
def return_dictonary(imgs, model):
    # Dictionary for storing marks of each papers
    my_dict = {'1a': [], '1b': [], '1c': [], '2a': [], '2b': [], '2c': [], '3a': [], '3b': [], '3c': [], '4a': [],
               '4b': [], '4c': [], '5a': [], '5b': [], '5c': [], '6a': [], '6b': [], '6c': [], '7a': [], '7b': [],
               '7c': [], '8a': [], '8b': [], '8c': [], '9a': [], '9b': [], '9c': [], '10a': [], '10b': [], '10c': [],
               '11a': [], '11b': [], '11c': [], '12a': [], '12b': [], '12c': []}

    for i in range(len(imgs)):
        img = imgs[i]
        dpi = (200, 200)
        img.info["dpi"] = dpi
        img_bytes = io.BytesIO()
        img.save(img_bytes, format='JPEG')
        img_bytes.seek(0)
        doc = Image(img_bytes)
        extracted_tables = doc.extract_tables(implicit_rows=False, min_confidence=50)
        orddict = extracted_tables[0].content
        img = PIL_Image.open(img_bytes)
        if len(orddict.keys()) == 5 and sum(len(value) for value in orddict.values()) == 65:
            # if the table recognition is correct, the no of rows will be 5
            paper_df = cell_extraction_classification_df_return(orddict, img, model)
            marks_for_main_dict = dataframe_postprocessing(paper_df)

            for key, value in zip(my_dict.keys(), marks_for_main_dict):
                my_dict[key].append(value)  # Adding values to dictionary

        else:  # if the table recognition is incorrect
            for key in my_dict.keys():
                my_dict[key].append(0)  # Adding values to dictionary
    return my_dict

In [9]:
import tensorflow as tf

In [10]:
model = tf.keras.models.load_model("MP_Latest_Model.h5")
image_list = []
dict_ret_df_lst = []